In [2]:
# NumPy, SciPy and Pandas
from scipy.spatial.distance import cdist
import numpy as np
import pandas as pd

# TSLEARN
from tslearn.preprocessing import TimeSeriesScalerMeanVariance
from tslearn.utils import to_time_series_dataset
from tslearn.clustering import KShape


In [3]:
"""
Perform the k-shape algorithm using the tslearn library. The function takes a dataframe of multiple time series (each
row is a time series) and returns the centroids as well as the cluster labels for each time series
"""
def doKshape(dataframe, dataframe_name, k, seed=3, max_iter=200, plot=False):
    df_scaled = TimeSeriesScalerMeanVariance(mu=0.0, std=1.0).fit_transform(dataframe.values)
    # dataframe.values will generate a 3d array (the third dimension being 1) so we convert it to a 2d array
    df_scaled = np.squeeze(df_scaled) # now is a 2d array

    sz = df_scaled.shape[1] # length of each time series
    ks = KShape(n_clusters=k, verbose=False, random_state=seed, max_iter=max_iter) # create the model
    y_pred = ks.fit_predict(df_scaled) # fit the data and generate the cluster labels
    
    if plot: # for each cluster generate a plot
        fig = plt.figure(figsize=(20, 10))
        for yi in range(k):
            plt.subplot(k, 1, 1 + yi)

            # for each time series in the scaled dataframe
            for xx in df_scaled[y_pred == yi]:
                plt.plot(xx.ravel(), "k-", alpha=.2)

            # add the centroid (in red) to the plot
            plt.plot(ks.cluster_centers_[yi].ravel(), "r-")
            plt.xlim(0, sz)
            plt.ylim(-4, 4)
            plt.title("Cluster %d" % (yi + 1), fontsize = 20)
        fig.suptitle("Dataset: {}".format(dataframe_name), fontsize = 25)

    # return the clusterlabels and the cluster centers
    return ks, y_pred
